#### Written by Gabriel Borges Pinheiro
[Github](github.com/gabrielbopi)

In [1]:
import numpy as np
#import scipy.stats as sps
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def db(X):
    return 10 * np.log10(np.abs(X))

In [3]:
def inv_db(X):
    return 10**(X/10)

In [4]:
def beta_channel(D):
    """
    Returns the 
    """
    #Band (LTE band 7)
    B = 2.6e9
    #Wavelength
    l = 3e8/B
    R = D
    #Linear pathloss
    L_lin =  l/(4*np.pi*R)**2
    beta = L_lin
    return beta


In [5]:
def rand_channel(D, N):
    """
    Returns the channel
    """
    N = 2
    beta = beta_channel(D)
    amp = np.sqrt(beta)/2
    h = (np.random.randn(N) + 1j*np.random.randn(N)) * amp
    return h


In [6]:
def compute_snr(D, N):
    h = rand_channel(D, N)
    #Nomalizing for precoding
    v = np.conj(h) / np.abs(h)
    #SNR_t=1:
    P_t = inv_db(10)
    sigma = inv_db(10)
    SNR_t = P_t/sigma**2
    SNR = np.abs(v.transpose() * h)**2 * SNR_t
    return SNR

In [7]:
def snr_average(mc,D, N):
    """
    The average precoded channel
    """
    samples_snr = np.array([])
    for i in range(mc):
        samples_snr = np.append(samples_snr, compute_snr(D, N))
    return np.mean(samples_snr)

In [8]:
#Distance between BS and UE [meter] inner cell
R = np.array([10,50,15])
#Number of Monte Carlo realizations
mc = 100
#Number of resolution for plotting
n = 30
#Band (LTE band 7)
B = 2.6e9

In [12]:
#Maximum range between BS and UE [meter] inner cell
D_max = 150
#Generate a random point for UE position (height of antenna BS = 15m)
R = np.array(np.append(np.sqrt(D_max)*np.random.randn(2), 15))
#Number of Monte Carlo realizations
mc = 100
#Number of resolution for plotting
n = 30
#Band (LTE band 7)
B = 2.6e9
#Number of array's antennas
N = 64

In [13]:
print('location: {}'.format(R))
D = np.linalg.norm(R)
db(snr_average(mc,D, N))

location: [ 6.63475302  0.51926098 15.        ]


-68.55305833068223